In [1]:
from src.data import player_data
from src.util import characters, display_progress
from src.transfer import ssbml_transfer_model
from tensorflow import keras
from tensorflow_addons.losses import SigmoidFocalCrossEntropy as Focal
import numpy as np
import pickle
import os

# Model with Pre-Trained Base

In [21]:
model = ssbml_transfer_model()

In [22]:
model.summary()

Model: "SSBML-Transfer-Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SSBML-Base-Model (Sequential (None, 512)               6537842   
_________________________________________________________________
head_densex2 (Sequential)    (None, 1)                 83329     
Total params: 6,621,171
Trainable params: 82,817
Non-trainable params: 6,538,354
_________________________________________________________________


# The Players

In [6]:
!ls data/player

Blynde	CuckDaddy  Lie0x  TCBL	gh0st  ixwonkr


In [7]:
# how many clips does a player have?
!ls data/player/Blynde/train | wc -l

4286


# The Player Data

Player Data can be found in data/player/\<player name\>

Nonplayer Data is taken from the large dataset data/character

In [8]:
# name of the player we want to train/test on
player_name = 'Blynde'

player_dir = os.path.join('data/player', player_name)
player_train_dir = os.path.join(player_dir, 'train')
player_test_dir = os.path.join(player_dir, 'test')

nonplayer_dir = 'data/character'
nonplayer_train_dir = os.path.join(nonplayer_dir, 'train')
nonplayer_test_dir = os.path.join(nonplayer_dir, 'test')

In [9]:
print(f'Training Data Located at: \n\t- {player_train_dir} \n\t- {nonplayer_train_dir} \n')
print(f'Testing Data Located at: \n\t- {player_test_dir} \n\t- {nonplayer_test_dir} \n')

Training Data Located at: 
	- data/player/Blynde/train 
	- data/character/train 

Testing Data Located at: 
	- data/player/Blynde/test 
	- data/character/test 



# Testing

In [25]:
# testing data
data_test_one_round = player_data(
    player_test_dir,
    nonplayer_test_dir,
    batch_size = 32,
    repeat = False,
#     ratio=.1,
)

score = model.evaluate(data_test_one_round, verbose=1)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

31/31 [==============================] - 3s 87ms/step - loss: 0.3176 - binary_accuracy: 0.8690

Test score: 0.318
Test accuracy: 87%


# Confusion Matrix

In [26]:
import seaborn as sn
import pandas as pd
from sklearn.metrics import confusion_matrix

In [30]:
num_batches = 1
data_conf = player_data(
    player_test_dir,
    nonplayer_test_dir,
    batch_size = 25,
    repeat = False,
#     ratio=.1,
)

In [31]:
batch_preds = []
batch_labels = []
i = 0

for xi, yi in data_conf:
    batch_preds.append(np.around(model.predict(xi)).astype(int).reshape((-1)))
    batch_labels.append(yi)
    
    # just in case
    i += 1
    if i > 50000:
        break
    
#     i+=1
#     display_progress(i, num_batches)
# display_progress(num_batches, num_batches)
    
pred = np.concatenate(batch_preds)
labels = np.concatenate(batch_labels)

conf_matrix = confusion_matrix(labels, pred, normalize='all')[::-1, ::-1] # reverse so true positive is top left, true negative is bottom right
conf_matrix = np.around(conf_matrix, 3)
conf_df = pd.DataFrame(conf_matrix, index=['player', 'nonplayer'], columns=['player', 'nonplayer'])

In [32]:
conf_df

,player,nonplayer
player,0.476,0.124
nonplayer,0.019,0.381


# Training

In [13]:
# training data
small_batches = player_data(
    player_train_dir,
    nonplayer_train_dir,
    repeat = True,
    batch_size = 32,
    ratio=.1,
)

# testing data
data_test = player_data(
    player_test_dir,
    nonplayer_test_dir,
    repeat = True,
    batch_size = 25,
    ratio=.1,
)

In [23]:
# Training Loop
model.fit(
    small_batches,
    epochs = 10,
    steps_per_epoch = 300,
    verbose = 1,
)

score = model.evaluate(data_test, verbose=0, steps=25)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

Epoch 1/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2568 - binary_accuracy: 0.8935
Epoch 2/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2358 - binary_accuracy: 0.9026
Epoch 3/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2383 - binary_accuracy: 0.9014
Epoch 4/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2360 - binary_accuracy: 0.8998
Epoch 5/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2367 - binary_accuracy: 0.9002
Epoch 6/10
300/300 [==============================] - 13s 45ms/step - loss: 0.2339 - binary_accuracy: 0.9020
Epoch 7/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2249 - binary_accuracy: 0.9070
Epoch 8/10
300/300 [==============================] - 13s 45ms/step - loss: 0.2266 - binary_accuracy: 0.9059
Epoch 9/10
300/300 [==============================] - 13s 44ms/step - loss: 0.2264 - binary_accuracy: 0.9028
Epoch 10/10
300/300

In [24]:
# testing data
data_test_one_round = player_data(
    player_test_dir,
    nonplayer_test_dir,
    batch_size = 32,
    repeat = False,
    ratio = .5
)

score = model.evaluate(data_test_one_round, verbose=1)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

45/45 [==============================] - 4s 89ms/step - loss: 0.2630 - binary_accuracy: 0.8938

Test score: 0.263
Test accuracy: 89%
